# Feature Matrix Creation

This notebook details the creation of the feature matrix that we will be using in our various algorithms to predict NHL fantasy performance. From a high level, we will be reading in the tables that we have created from the NHL API.

In the feature matrix, there is a row for every player for each game they have played over the last 5 seasons. We use previous performance averages over various time frames in order to gauge player performance accurately.

The first part of this is to read in the 5 tables that we have created in our DB.

In [1]:
import numpy as np
import pandas as pd

# sort this one by player_id, season, then game_id so we can just look back for the previous game aggregations
player_stats_by_game = pd.read_csv('player_stats_by_game.csv').sort_values(by=['player_id', 'season', 'game_id']).reset_index(drop=True)
players = pd.read_csv('players.csv')
season_team_stats = pd.read_csv('season_team_stats.csv')
team_stats_per_game = pd.read_csv('team_stats_per_game.csv').sort_values(by=['teamid', 'season', 'gameid']).reset_index(drop=True)
teams = pd.read_csv('teams.csv')

player_stats_by_game['skaterStatsevenTimeOnIce'] = pd.to_datetime(player_stats_by_game['skaterStatsevenTimeOnIce'], format = "%M:%S")
player_stats_by_game['skaterStatsevenTimeOnIce'] = player_stats_by_game['skaterStatsevenTimeOnIce'].dt.minute * 60 + player_stats_by_game['skaterStatsevenTimeOnIce'].dt.second

player_stats_by_game['skaterStatspowerPlayTimeOnIce'] = pd.to_datetime(player_stats_by_game['skaterStatspowerPlayTimeOnIce'], format = "%M:%S")
player_stats_by_game['skaterStatspowerPlayTimeOnIce'] = player_stats_by_game['skaterStatspowerPlayTimeOnIce'].dt.minute * 60 + player_stats_by_game['skaterStatspowerPlayTimeOnIce'].dt.second


In [4]:
# player_stats_by_game.head(10)
team_stats_per_game.head(85)

,gameid,season,homeOrAway,teamSkaterStatsgoals,teamSkaterStatspim,teamSkaterStatsshots,teamSkaterStatspowerPlayPercentage,teamSkaterStatspowerPlayGoals,teamSkaterStatspowerPlayOpportunities,teamSkaterStatsfaceOffWinPercentage,teamSkaterStatsblocked,teamSkaterStatstakeaways,teamSkaterStatsgiveaways,teamSkaterStatshits,teamid
0,12,2015,H,1,8,21,25,1,4,57,6,8,4,17,1
1,22,2015,A,3,12,25,33,1,3,44,17,4,4,32,1
2,37,2015,H,1,4,23,50,1,2,49,8,3,9,13,1
3,58,2015,H,1,4,32,17,1,6,37,7,5,9,35,1
4,75,2015,A,2,9,24,0,0,3,44,18,4,10,30,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,1203,2015,H,2,13,22,0,0,1,55,11,1,2,19,1
81,1222,2015,H,5,6,36,0,0,6,46,12,6,2,7,1
82,10,2016,A,1,16,24,0,0,2,46,16,8,15,18,1
83,19,2016,A,2,8,34,0,0,3,48,12,7,2,27,1


In [80]:
feature_columns = ['player_id', 'game_id', 'season', 'homeOrAway', 'goals_1', 'goals_5', 'goals_10', 'goals_20', 'goals_season', 
                   'assists_1', 'assists_5', 'assists_10', 'assists_20', 'assists_season',
                   'shots_1', 'shots_5', 'shots_10', 'shots_20', 'shots_season', 
                   'hits_1', 'hits_5', 'hits_10', 'hits_20', 'hits_season', 
                   'blocks_1', 'blocks_5', 'blocks_10', 'blocks_20', 'blocks_season', 
                   'es_toi_1', 'es_toi_5', 'es_toi_10', 'es_toi_20', 'es_toi_season', 
                  'pp_toi_1', 'pp_toi_5', 'pp_toi_10', 'pp_toi_20', 'pp_toi_season', 
                  'pp_goals_1', 'pp_goals_5', 'pp_goals_10', 'pp_goals_20', 'pp_goals_season', 
                  'pp_assists_1', 'pp_assists_5', 'pp_assists_10', 'pp_assists_20', 'pp_assists_season', 
                  'takeaways_1', 'takeaways_5', 'takeaways_10', 'takeaways_20', 'takeaways_season', 
                  'giveaways_1', 'giveaways_5', 'giveaways_10', 'giveaways_20', 'giveaways_season', 
                  'plus_minus_1', 'plus_minus_5', 'plus_minus_10', 'plus_minus_20', 'plus_minus_season', 
                  'opp_goals_against_1', 'opp_goals_against_5', 'opp_goals_against_10', 'opp_goals_against_20', 'opp_goals_against_season', 
                  'sh_goals_1', 'sh_goals_5', 'sh_goals_10', 'sh_goals_20', 'sh_goals_season', 
                   'sh_assists_1', 'sh_assists_5', 'sh_assists_10', 'sh_assists_20', 'sh_assists_season', 'dfs_points']

periods = [1, 5, 10, 20]

agg_cols = {
            'assists' : 'skaterStatsassists', 
            'goals' : 'skaterStatsgoals', 
            'shots' : 'skaterStatsshots', 
            'hits' : 'skaterStatshits', 
            'pp_goals' : 'skaterStatspowerPlayGoals',
            'takeaways' : 'skaterStatstakeaways', 
            'giveaways' : 'skaterStatsgiveaways', 
            'es_toi' : 'skaterStatsevenTimeOnIce', 
            'pp_toi' : 'skaterStatspowerPlayTimeOnIce',
            'blocks' : 'skaterStatsblocked', 
            'pp_assists' : 'skaterStatspowerPlayAssists', 
            'plus_minus' : 'skaterStatsplusMinus', 
            'sh_goals' : 'skaterStatsshortHandedGoals',
            'sh_assists' : 'skaterStatsshortHandedAssists'
           }

In [110]:
rows = []
categories = ['goals', 'assists', 'shots', 'hits', 'blocks', 'es_toi', 'pp_toi', 'pp_goals', 'pp_assists',
              'takeaways', 'giveaways', 'plus_minus', 'sh_goals', 'sh_assists']

start_index = 0
season_index = 0
periods = [1, 5, 10, 20]

for i in range(0, len(player_stats_by_game)):
    if (player_stats_by_game.iloc[start_index, 0] != player_stats_by_game.iloc[i, 0]):
        start_index = i
    
    if (player_stats_by_game.iloc[season_index, 2] != player_stats_by_game.iloc[i, 2]):
        season_index = i
        print(season_index)
    
    game_history = i - start_index
    season = player_stats_by_game.iloc[i, 2]
    
    dict1 = {}
    
    dict1['player_id'] = player_stats_by_game.iloc[i, 0]
    dict1['game_id'] = player_stats_by_game.iloc[i, 1]
    dict1['season'] = season
    dict1['homeOrAway'] = player_stats_by_game.iloc[i, 3]
    
    if dict1['homeOrAway'] == 'H':
        opp_homeOrAway = 'A'
    else:
        opp_homeOrAway = 'H'
    
    for category in categories:
        for period in periods:
            if game_history >= period:
                dict1[category + "_" + str(period)] = player_stats_by_game[agg_cols[category]].iloc[(i-period):i].mean()
            else:
                dict1[category + "_" + str(period)] = None
        
        if season_index == i:
            dict1[category + "_season"] = None
        else:
            dict1[category + "_season"] = player_stats_by_game[agg_cols[category]].iloc[season_index:i].mean()
            
    dict1['dfs_points'] = (8.5 * player_stats_by_game[agg_cols['goals']].iloc[i]) + (5 * player_stats_by_game[agg_cols['assists']].iloc[i]) + (1.5 * player_stats_by_game[agg_cols['shots']].iloc[i]) + (1.3 * player_stats_by_game[agg_cols['blocks']].iloc[i]) + (2 * (player_stats_by_game[agg_cols['sh_goals']].iloc[i] + player_stats_by_game[agg_cols['sh_assists']].iloc[i]))
    
    bonus_categories = ['goals', 'shots', 'blocks', 'points']
    
    for bonus in bonus_categories:
        if bonus == 'shots':
            if player_stats_by_game[agg_cols[bonus]].iloc[i] >= 5:
                dict1['dfs_points'] += 3
        elif bonus == 'points':
            if (player_stats_by_game[agg_cols['goals']].iloc[i] + player_stats_by_game[agg_cols['assists']].iloc[i]) >= 3:
                dict1['dfs_points'] += 3
        else:
            if player_stats_by_game[agg_cols[bonus]].iloc[i] >= 3:
                dict1['dfs_points'] += 3
        
    ## Find opponent statistics'

#     opp_game = team_stats_per_game.loc[(team_stats_per_game['gameid'] == dict1['game_id']) & (team_stats_per_game['season'] == dict1['season']) & (team_stats_per_game['homeOrAway'] == opp_homeOrAway)]
#     opp_game_index = team_stats_per_game.index[(team_stats_per_game['gameid'] == dict1['game_id']) & (team_stats_per_game['season'] == dict1['season']) & (team_stats_per_game['homeOrAway'] == opp_homeOrAway)].tolist()[0]
#     opp_team_id = int(opp_game['teamid'])
#     opp_season_start = min(team_stats_per_game.index[(team_stats_per_game['season'] == dict1['season']) & (team_stats_per_game['teamid'] == opp_team_id)].tolist())

#     opp_games = opp_game_index - opp_season_start
#     for period in periods:
#         if opp_games >= period:
#             dict1[]
    
    rows.append(dict1)

df = pd.DataFrame(rows)


df.to_csv('feature_matrix.csv')


79
161
183
277
351
433
513
645
717
794
865
945
1020
1090
1152
1220
1271
1293
1389
1439
1521
1600
1647
1719
1789
1871
1953
2033
2115
2181
2245
2318
2400
2482
2554
2636
2716
3122
3194
3207
3288
3334
3484
3565
3629
3708
3790
3807
4009
4062
4144
4206
4354
4436
4514
4588
4670
4749
4835
4917
4929
4993
5075
5148
5149
5231
5313
5392
5455
5536
5577
5656
5734
5783
5845
5913
5915
5969
6025
6071
6150
6228
6289
6370
6442
6520
6601
6665
6706
6785
6864
6917
6998
7051
7133
7213
7293
7375
7395
7459
7516
7592
7671
7752
7780
7936
8008
8071
8149
8230
8241
8321
8403
8453
8521
8563
8623
8649
8753
8789
8858
8939
9015
9089
9138
9201
9265
9311
9386
9454
9529
9605
9663
9745
9825
9904
9952
10007
10098
10177
10254
10311
10371
10446
10461
10468
10606
10675
10717
10778
10842
10924
10988
11035
11036
11111
11189
11268
11341
11418
11483
11565
11643
11718
11721
11797
11845
11912
11978
12028
12147
12228
12287
12348
12421
12447
12581
12650
12707
12788
12800
12882
12934
12986
13063
13130
13212
13271
13341
13344
13395
1345

85801
85861
85947
86027
86088
86162
86227
86267
86268
86336
86338
86409
86481
86559
86641
86712
86768
86841
86889
86961
87030
87108
87189
87248
87321
87386
87468
87547
87621
87694
87760
87821
87843
87921
87998
88078
88160
88223
88301
88380
88454
88524
88590
88666
88743
88822
88895
88961
89026
89096
89173
89247
89311
89391
89470
89549
89621
89692
89767
89846
89925
90006
90072
90148
90230
90308
90381
90432
90517
90549
90625
90705
90784
90855
90896
90978
91027
91099
91181
91214
91288
91317
91353
91432
91504
91583
91642
91716
91763
91825
91882
91933
91998
92046
92068
92149
92230
92309
92391
92451
92539
92614
92670
92749
92812
92870
92908
92947
93020
93083
93162
93223
93247
93328
93377
93450
93531
93602
93684
93764
93830
93904
93975
93977
93982
94063
94145
94224
94304
94372
94454
94528
94594
94671
94741
94781
94808
94890
94972
95052
95134
95203
95264
95324
95355
95371
95392
95395
95474
95544
95622
95698
95726
95756
95838
95907
95982
96041
96044
96046
96060
96142
96224
96302
96382
96450
9652

148322
148397
148476
148555
148629
148685
148767
148849
148925
148988
149056
149137
149186
149262
149338
149386
149464
149545
149625
149707
149775
149797
149808
149876
149938
149940
150021
150081
150135
150217
150287
150296
150325
150359
150390
150432
150514
150596
150660
150724
150787
150861
150936
150981
151061
151137
151218
151284
151285
151286
151288
151347
151413
151481
151560
151641
151720
151800
151864
151870
151925
151987
152048
152066
152076
152116
152190
152247
152251
152324
152401
152483
152552
152561
152565
152630
152694
152759
152823
152899
152975
153053
153095
153124
153175
153183
153186
153189
153271
153350
153426
153471
153486
153496
153563
153623
153687
153696
153740
153816
153894
153932
154012
154089
154170
154234
154313
154393
154472
154553
154624
154705
154782
154860
154942
154987
154988
155016
155059
155063
155135
155215
155290
155372
155441
155443
155508
155538
155555
155589
155647
155725
155800
155857
155885
155954
156031
156107
156160
156219
156285
156306
156363

200809
200891
200959
200976
201023
201031
201035
201045
201077
201121
201123
201153
201198
201272
201317
201397
201479
201558
201637
201706
201761
201825
201831
201855
201882
201885
202052
202095
202103
202113
202138
202171
202190
202219
202276
202348
202414
202427
202494
202572
202653
202721
202725
202795
202802
202808
202818
202865
202916
202966
203010
203025
203045
203052
203082
203085
203087
203090
203137
203254
203257
203259
203292
203363
203427
203429
203490
203503
203539
203554
203555
203587
203634
203636
203692
203751
203802
203874
203910
203956
204022
204083
204163
204200
204225
204295
204325
204342
204387
204389
204392
204395
204402
204413
204434
204484
204491
204495
204528
204536
204581
204587
204598
204599
204606
204617
204651
204689
204690
204700
204708
204712
204713
204754
204803
204806
204857
204916
204996
205064
205073
205149
205217
205297
205366
205400
205418
205437
205486
205549
205631
205691
205759
205829
205885
205961
205966
206026
206086
206140
206224
206230
206287

In [112]:
df.head(1000)

,player_id,game_id,season,homeOrAway,goals_1,goals_5,goals_10,goals_20,goals_season,assists_1,...,sh_goals_5,sh_goals_10,sh_goals_20,sh_goals_season,sh_assists_1,sh_assists_5,sh_assists_10,sh_assists_20,sh_assists_season,dfs_points
0,8448208,20,2015,H,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5
1,8448208,35,2015,A,2.0,NaN,NaN,NaN,2.000000,0.0,...,NaN,NaN,NaN,0.000000,0.0,NaN,NaN,NaN,0.0,3.0
2,8448208,41,2015,A,0.0,NaN,NaN,NaN,1.000000,0.0,...,NaN,NaN,NaN,0.000000,0.0,NaN,NaN,NaN,0.0,8.0
3,8448208,55,2015,H,0.0,NaN,NaN,NaN,0.666667,1.0,...,NaN,NaN,NaN,0.000000,0.0,NaN,NaN,NaN,0.0,29.5
4,8448208,68,2015,H,2.0,NaN,NaN,NaN,1.000000,1.0,...,NaN,NaN,NaN,0.000000,0.0,NaN,NaN,NaN,0.0,9.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,8465009,832,2016,H,0.0,0.2,0.1,0.10,0.080000,0.0,...,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,16.1
996,8465009,877,2016,A,1.0,0.4,0.2,0.15,0.098039,0.0,...,0.2,0.1,0.05,0.019608,0.0,0.0,0.0,0.0,0.0,4.3
997,8465009,893,2016,A,0.0,0.2,0.2,0.15,0.096154,0.0,...,0.2,0.1,0.05,0.019231,0.0,0.0,0.0,0.0,0.0,11.5
998,8465009,899,2016,A,1.0,0.4,0.3,0.20,0.113208,0.0,...,0.2,0.1,0.05,0.018868,0.0,0.0,0.0,0.0,0.0,12.5


In [4]:
import numpy as np
import pandas as pd

fet = pd.read_csv('feature_matrix_w_position.csv')

fet.head(10)

,Unnamed: 0,player_id,game_id,season,homeOrAway,goals_1,goals_5,goals_10,goals_20,goals_season,...,sh_goals_season,sh_assists_1,sh_assists_5,sh_assists_10,sh_assists_20,sh_assists_season,dfs_points,primaryPositioncode,primaryPositionname,primaryPositiontype
0,0,8448208,20,2015,H,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,21.5,NaN,NaN,NaN
1,1,8448208,35,2015,A,2.0,NaN,NaN,NaN,2.000000,...,0.0,0.0,NaN,NaN,NaN,0.0,3.0,NaN,NaN,NaN
2,2,8448208,41,2015,A,0.0,NaN,NaN,NaN,1.000000,...,0.0,0.0,NaN,NaN,NaN,0.0,8.0,NaN,NaN,NaN
3,3,8448208,55,2015,H,0.0,NaN,NaN,NaN,0.666667,...,0.0,0.0,NaN,NaN,NaN,0.0,29.5,NaN,NaN,NaN
4,4,8448208,68,2015,H,2.0,NaN,NaN,NaN,1.000000,...,0.0,0.0,NaN,NaN,NaN,0.0,9.3,NaN,NaN,NaN
5,5,8448208,83,2015,A,0.0,0.8,NaN,NaN,0.800000,...,0.0,0.0,0.0,NaN,NaN,0.0,4.5,NaN,NaN,NaN
6,6,8448208,97,2015,A,0.0,0.4,NaN,NaN,0.666667,...,0.0,0.0,0.0,NaN,NaN,0.0,13.0,NaN,NaN,NaN
7,7,8448208,113,2015,A,1.0,0.6,NaN,NaN,0.714286,...,0.0,0.0,0.0,NaN,NaN,0.0,24.0,NaN,NaN,NaN
8,8,8448208,128,2015,H,1.0,0.8,NaN,NaN,0.750000,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN
9,9,8448208,184,2015,A,0.0,0.4,NaN,NaN,0.666667,...,0.0,0.0,0.0,NaN,NaN,0.0,1.5,NaN,NaN,NaN
